# Using CW-Analyzer for CPA Attack

This tutorial will take you through a complete attack on a software AES implementation. The specific implementation being attacked is a well-known AES implementation written in C, which is likely to be similar to other implementations used by proprietary systems.

## Capturing Power Traces

### Setup

We'll use some helper scripts to make setup and programming easier. If you're using an XMEGA or STM (CWLITEARM) target, binaries with the correct should be setup for you:

In [1]:
import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope)

In [2]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [3]:
fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-cwlitearm.hex"
prog = cw.programmers.STM32FProgrammer
#prog = cw.programmers.XMEGAProgrammer

In [4]:
cw.programTarget(scope, prog, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 5879 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5879 bytes


In addition, before we capture our traces, we'll need to create a ChipWhipserer project, since that's what Analyzer expects for an input:

In [5]:
project = cw.createProject("projects/Tutorial_B5.cwp", overwrite = True)

And we can get the class used to hold our traces by:

In [6]:
tc = project.getTraceFormat()

### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and our new trace into our trace class.

In [7]:
#Capture Traces
from tqdm import tqdm
import numpy as np
import time

ktp = cw.ktp.Basic(target=target)

N = 50  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    #traces.append(scope.getLastTrace())
    tc.addTrace(scope.getLastTrace(), text, "", key)

Capturing traces: 100%|██████████| 50/50 [00:07<00:00,  6.57it/s]


Now that we have our traces, we need to tell the project that the traces are loaded and add them to the project's trace manager.

In [8]:
tc._isloaded = True
project.traceManager().appendSegment(tc)

If you'd like, you can also save the project for later analysis (this closes the project, so if you run this block you'll need to reopen it in the next section):

In [9]:
from datetime import datetime
import copy

starttime = datetime.now()
prefix = starttime.strftime('%Y.%m.%d-%H.%M.%S') + "_"
tc.config.setConfigFilename(project.datadirectory + "traces/config_" + prefix + ".cfg")
tc.config.setAttr("prefix", prefix)
tc.config.setAttr("date", starttime.strftime('%Y-%m-%d %H:%M:%S'))
tc.closeAll()
project.save()

We're now done with the ChipWhisperer hardware, so we should disconnect from the scope and target:

In [10]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Analysis

If you saved in the last part (or if you're continuing from where you left off), you'll need to reload the project:

In [1]:
import chipwhisperer as cw
project = cw.openProject("projects/Tutorial_B5.cwp")
tm = project.traceManager()

Now that we have our traces, we can begin our attack! We'll start off by setting up our attack by creating an attack object:

In [2]:
attack = cw.cpa(tm)

And then setting up the leakage model (more on this in the following CPA tutorials):

In [3]:
leak_model = cw.AES128(cw.aes128leakage.SBox_output)

attack.setLeakModel(leak_model)

With that done, we can run our attack:

In [4]:
attack_results = attack.processTraces()

Once you see the above block complete, all the heavylifting is done! All that's left is to actually look at the data. Everything important is contained in the `attack_results` class that `attack.processTraces()` returned.

We can find the max correlation for every one of the subkey by calling `stats.findMaximums()`, which returns a list of the subkeys, the point location of the max if calculated, and the correlation (which is a value between 0 and 1 that effectively tells us how well our guess fit the data). 


Note the "point location of the max" is normally not calculated/tracked, and thus returns as a 0. Using the pandas library lets us print them nicely in a DataFrame. We have to transpose the frame to get our expected orientation:

In [5]:
import pandas as pd
stat_data = attack_results.findMaximums()
df = pd.DataFrame(stat_data).transpose()
print(df.head())

                             0                             1   \
0   [43, 0, 0.9258841708147422]  [126, 0, 0.9128793939621457]   
1  [227, 0, 0.6051039178535276]  [203, 0, 0.6219971007694665]   
2   [76, 0, 0.5971356939139717]   [58, 0, 0.6207506404279537]   
3  [210, 0, 0.5918332721357764]   [68, 0, 0.5998286341123509]   
4   [95, 0, 0.5849159241830654]  [226, 0, 0.5933977404942192]   

                             2                             3   \
0   [21, 0, 0.9347054273985043]   [22, 0, 0.9126116633809283]   
1   [77, 0, 0.6456673288412471]  [199, 0, 0.6191046565331646]   
2   [90, 0, 0.6432670319870437]  [208, 0, 0.6055107168649803]   
3  [153, 0, 0.6219110408775452]  [149, 0, 0.6035157825872411]   
4   [33, 0, 0.6087746604100835]  [143, 0, 0.5990593169666122]   

                             4                             5   \
0   [40, 0, 0.8651864171424126]  [174, 0, 0.8960921245634439]   
1  [117, 0, 0.6419120355383365]  [124, 0, 0.6049193074907651]   
2   [36, 0, 0.615994847

Even better, we can use the `.style` method to customize this further. This also lets us chain formatting functions. For example, we can remove the extra 0 and clean up the data. Since we know the correct key, we can even do things like printing the key in a different colour! 

You can do lots of formatting thanks to the pandas library! Check out https://pandas.pydata.org/pandas-docs/stable/style.html for more details.

In [6]:
key = project.traceManager().getKnownKey(0)
def format_stat(stat):
    return str("{:02X}<br>{:.3f}".format(stat[0], stat[2]))

def color_corr_key(row):
    global key
    ret = [""] * 16
    for i,bnum in enumerate(row):
        if bnum[0] == key[i]:
            ret[i] = "color: red"
        else:
            ret[i] = ""
    return ret

df.head().style.format(format_stat).apply(color_corr_key, axis=1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,2B0.926,7E0.913,150.935,160.913,280.865,AE0.896,D20.871,A60.869,AB0.899,F70.892,150.895,880.896,090.843,CF0.847,4F0.867,3C0.888
1,E30.605,CB0.622,4D0.646,C70.619,750.642,7C0.605,F40.666,830.638,520.626,C70.694,DB0.659,8D0.641,D80.659,B20.663,700.622,F30.656
2,4C0.597,3A0.621,5A0.643,D00.606,240.616,5A0.592,000.609,C50.608,440.618,8F0.653,F90.644,8F0.633,8A0.658,C50.621,D90.621,9E0.604
3,D20.592,440.600,990.622,950.604,FD0.612,E30.589,B80.600,690.598,2E0.604,3D0.645,0C0.638,B90.605,FE0.628,C20.615,5B0.613,C00.599
4,5F0.585,E20.593,210.609,8F0.599,AB0.607,B10.585,2E0.593,480.593,AD0.596,250.611,490.628,3D0.603,520.618,3B0.603,920.603,2F0.597


You should see red numbers printed at the top of a table. Congratulations, you've now completed a successful CPA attack against AES!

Next, we'll look at how we can use some of Analyzer's other features to improve the attack process, as well as better interpret the data we have.

### Reporting Intervals

When we ran `attack.processTraces()`, we processed all of the traces before getting any information back. While this works okay for shorter attacks like this, for longer ones it can helpful to get feedback during the attack. This can be done by creating a callback function and passing it to `attack.processTraces()`. This function is called each time we pass our `attack.setReportingInterval()` (default 25 traces) and has access to everything a normal python function does.

Let's use this to update our table every 10 traces. Most of this is just putting our existing code into the callback function. We also need use the `clear_output` function to clear the table, as well as `display()` to actually get it to show up:

In [7]:
from IPython.display import clear_output
import numpy as np
        
def stats_callback():
    attack_results = attack.getStatistics()
    attack_results.setKnownkey(key)
    stat_data = attack_results.findMaximums()
    df = pd.DataFrame(stat_data).transpose()
    clear_output(wait=True)
    display(df.head().style.format(format_stat).apply(color_corr_key,axis=1))
    
attack.setReportingInterval(10)
attack_results = attack.processTraces(stats_callback)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,2B0.926,7E0.913,150.935,160.913,280.865,AE0.896,D20.871,A60.869,AB0.899,F70.892,150.895,880.896,090.843,CF0.847,4F0.867,3C0.888
1,E30.605,CB0.622,4D0.646,C70.619,750.642,7C0.605,F40.666,830.638,520.626,C70.694,DB0.659,8D0.641,D80.659,B20.663,700.622,F30.656
2,4C0.597,3A0.621,5A0.643,D00.606,240.616,5A0.592,000.609,C50.608,440.618,8F0.653,F90.644,8F0.633,8A0.658,C50.621,D90.621,9E0.604
3,D20.592,440.600,990.622,950.604,FD0.612,E30.589,B80.600,690.598,2E0.604,3D0.645,0C0.638,B90.605,FE0.628,C20.615,5B0.613,C00.599
4,5F0.585,E20.593,210.609,8F0.599,AB0.607,B10.585,2E0.593,480.593,AD0.596,250.611,490.628,3D0.603,520.618,3B0.603,920.603,2F0.597


A default jupyter callback is also available:

In [ ]:
import chipwhisperer as cw
cb = cw.getJupyterCallback(attack)
attack_results = attack.processTraces(cb)

Here we used a reporting interval of 10 traces. Depending on the attack and what you want to learn from it, you may want to use higher or lower values: in general reporting less often is faster, but more frequent reporting can allow you to end a long attack early. More frequent reporting also increases the resolution of some plot data (which we will look at next).

### Plot Data

Analyzer also includes a module to create plots to help you interpret the data. These act on one subkey at a time and return some data that we can plot using bokeh (or your graphing module of choice). Let's start by grabbing the class that does all the calculations:

In [ ]:
plot_data = cw.analyzerPlots(attack_results)

#### Output Vs. Time

We'll start by looking at the Output Vs. Time module, which will allow us to plot correlation of our guesses in time. This is useful for finding exactly where the operations we're attacking are. Like in previous tutorials, we'll use bokeh to plot the data we get back.

The method we're interested in is `getPlotData(bnum)`, which returns in a list: `[xrange, correct_key, incorrect_key_data, incorrect_key_data]` for the position `bnum` passed to it. The method returns two sets of incorrect key data because one is for the key guesses below the correct one, and the other is for guesses above the correct one.

We'll start by just looking at the 0th subkey. Once we get this data back we'll plot the correct key in red, and the rest in green.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

ret = plot_data.outputVsTime(0)

output_notebook()
p = figure()
p.line(ret[0], ret[2], line_color='green')
p.line(ret[0], ret[3], line_color='green')

p.line(ret[0], ret[1], line_color='red')
show(p)

You should see some distinctive red spikes in your plot. The largest of these is where the sbox lookup is actually happening (the smaller ones are typically other AES operations that move the sbox data around).

Let's repeat this for all the subkeys. This is quite a bit more data to plot, so give it a few seconds:

In [ ]:
"""rets = []
for i in range(0, 16):
    rets.append(plot_data.outputVsTime(i))

p = figure()
for ret in rets:
    p.line(ret[0], ret[2], line_color='green')
    p.line(ret[0], ret[3], line_color='green')
    
for ret in rets:
    p.line(ret[0], ret[1], line_color='red')

show(p)"""

This information can be useful in many ways. For example, you can probably see the first 16 spikes that make up the sbox lookup are a small portion of the total trace length. If we ever needed to rerun the attack, we could capture a much smaller number of samples and speed up analysis significantly!

#### PGE vs. Traces

The next data we'll look at is a plot of partial guessing entropy (PGE) vs. the number of traces. As mentioned before, PGE is just how many spots away from the top the actual subkey is in our table of guesses. For example, if there are 7 subkey guesses that have a higher correlation than the actual subkey, the subkey has a PGE of 7.

This plot is useful for seeing how many traces were needed to actually break the AES implementation. Keep in mind, however, that the resolution of the plot is determined by the reporting interval (also note that `attack_results.findMaximums()` must be called in the callback function). In our case, we have a reporting interval of 10, so we'll have a resolution of 10 traces.

This module's `getPlotData()` is similar to the previous plot in that it takes `bnum` as an argument and returns a list of `[xrange, PGE]`. Plotting this data is quite a bit faster than the previous example, we'll just plot all 16 of the `bnum` now.

In [ ]:
p = figure()

for bnum in range(16):
    ret = plot_data.pgeVsTrace(bnum)
    p.line(ret[0], ret[1], line_color='red')
show(p)

You should see a number of lines that start off with high values, then rapidly drop off. You may notice that we broke the AES implementation without needing to use all of our traces. 

Even though we may have broken the AES implementation in fewer traces, we may not want to reduce how many traces we capture. Remember that, while we know the key here, for a real attack we won't and therefore must use the correlation to determine when we've broken a key. Our next plot will help us to determine how feesible capturing fewer traces is.

#### Correlation vs. Traces

The last plot we'll take a look at is correlation vs the number of traces. Like with PGE vs. Traces, this plot's resolution is determined by the reporting interval (10 in our case). One again, this is a plot with a lot of data, so we'll start of by just plotting one subkey:

This module's `getPlotData()` returns a list of `[xrange, [data_for_kguess]]`, so we'll need to plot each guess for each subkey. Like before, we'll do the plot for the correct subkey in red and the rest in green.

In [ ]:
ret = plot_data.corrVsTrace(0)
p = figure()
for i in range(255):
    if i == key[0]:
        p.line(ret[0], ret[1][i], line_color='red')
    else:
        p.line(ret[0], ret[1][i], line_color='green')
        
show(p)

As you can see, all the subkey guesses start of with large correlations, but all of them except for the correct guess quickly drop off. If you didn't know the key, at what point would you be sure that the guess with the highest correlation was actually the correct subkey?

Let's continue and plot all of the subkeys (give this one some time):

In [ ]:
"""p = figure()
for bnum in range(16):
    ret = plot_data.corrVsTrace(bnum)
    for i in range(255):
        if i == key[bnum]:
            p.line(ret[0], ret[1][i], line_color='red')
        else:
            p.line(ret[0], ret[1][i], line_color='green')
            
show(p)"""

Like in the first plot, you should see the red lines remain high while the green ones drop off. At what point would you be sure that you've broken all the subkeys? Is it higher than when all of the PGE lines reached zero?

## Conclusion

You should now have completed a successful CPA attack and learned about some on Analyzer's features for improving your attack! 

You can move onto more advanced tutorials, especially showing you how the actual attack works when performed manually (Tutorial B6). This tutorial also utilized tiny-AES128-C for Arm targets, which uses the same operations as the XMEGA target. A later tutorial will preform this attack on a more typical 32 bit AES implementation.

## Tests

In [ ]:
key = project.traceManager().getKnownKey(0)
recv_key = [kguess[0][0] for kguess in attack_results.findMaximums()]
assert (key == recv_key).all(), "Failed to recover encryption key\nGot: {}\nExpected: {}".format(recv_key, key)

In [ ]:
assert (attack_results.pge == [0]*16), "PGE for some bytes not zero: {}".format(attack_results.pge)

In [ ]:
max_corrs = [kguess[0][2] for kguess in attack_results.findMaximums()]
assert (np.all([corr > 0.75 for corr in max_corrs])), "Low correlation in attack (corr <= 0.75): {}".format(max_corrs)